In [17]:
import torch
import cv2
import mediapipe as mp
import time
import pandas as pd
import numpy as np
from collections import deque
import winsound

In [15]:
fall_detect_model = torch.jit.load("../Final_model/model_script.pt")

print(fall_detect_model)
print(fall_detect_model.model)
print(fall_detect_model.code)

RecursiveScriptModule(
  original_name=FallDetectModel
  (model): RecursiveScriptModule(original_name=LSTM)
  (output): RecursiveScriptModule(original_name=Linear)
)
RecursiveScriptModule(original_name=LSTM)
def forward(self,
    inputs: Tensor) -> Tensor:
  output = self.output
  model = self.model
  _0 = torch.slice((model).forward(inputs, ), 0, 0, 9223372036854775807)
  input = torch.slice(torch.select(_0, 1, -1), 1, 0, 9223372036854775807)
  return (output).forward(input, )



In [ ]:
MAX_FRAMES_LEN = 300
MAX_FRAMES_NUM = 298

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=0,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    smooth_landmarks=True
)
mp_drawing = mp.solutions.drawing_utils

# 중요 관절 인덱스
important_landmarks = [
    0, 11, 12, 13, 14, 15, 16,
    23, 24, 25, 26, 29, 30
]

window = deque(maxlen = ((MAX_FRAMES_LEN // 3) * len(important_landmarks)))

test_df = pd.DataFrame(columns = ["frame", "landmark_id", "x", "y", "z"])

for frame in np.arange(1, 301):
    for landmark in important_landmarks:
        test_df.loc[len(test_df)] = [frame, landmark, np.nan, np.nan, np.nan]

# 웹캠에서 영상 받기
cap = cv2.VideoCapture(0)  # 기본 카메라 사용

frame_count = 0
processed_frame = 1
start_time = time.time()
predict_time = start_time

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    current_time = time.time()  
    elapsed = current_time - start_time
    predict_elapsed = current_time - predict_time

    if elapsed >= 0.1:
        start_time = current_time
        # MediaPipe에 넣기 위해 RGB로 변환
        frame = cv2.resize(frame, (480, 360))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        # 관절 데이터 추출 및 표시
        if results.pose_landmarks:
            for idx in important_landmarks:
                landmark = results.pose_landmarks.landmark[idx]
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

        if results.pose_landmarks:
            for idx in important_landmarks:
                landmark = results.pose_landmarks.landmark[idx]
                window.append((landmark.x, landmark.y, landmark.z))

        cv2.imshow('Real-Time Pose Landmarks', frame)

        # Q 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    if predict_elapsed >= 5:
        test_df2 = test_df.copy(deep = True)

        j = 1

        for i in range(len(window)):
            test_df2.loc[j - 1, ['x', 'y', 'z']] = [window[i][0], window[i][1], window[i][2]]
            j += 1

            if j % 13 == 1:
                j += 26

        test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)
        test_df2 = test_df2.sort_values(by = ["frame", "landmark_id"]).reset_index(drop = True)

        # frame 오름차순, source_index 우선 정렬
        test_df2 = test_df2.sort_values(by=["frame"])

        # 피벗: (frame, source_index) 기준, landmark_id 별 x/y/z를 칼럼으로
        df_pivot = test_df2.pivot(index=["frame"], columns="landmark_id", values=["x", "y", "z"])

        # 다중 인덱스 컬럼을 단일 열로 변환: ex) ('x', 11.0) -> x_11
        df_pivot.columns = [f"{coord}_{int(lid)}" for coord, lid in df_pivot.columns]

        # 인덱스 복구
        df_pivot.reset_index(inplace=True)
                            
        # 다시 source_index 기준으로 묶고, 그 안에서 frame 오름차순 정렬 (보장용)
        df_pivot = df_pivot.sort_values(by=["frame"]).reset_index(drop=True)
        
        X_tensor = torch.tensor(df_pivot.values, dtype = torch.float32)
        X_tensor = X_tensor.unsqueeze(0)

        output = torch.sigmoid(fall_detect_model(X_tensor))
        output = (output > 0.5).int()
        
        if output == 1:
            winsound.Beep(440, 1000)
        print(output)

        predict_time = current_time


cap.release()
cv2.destroyAllWindows()

C:\Users\이민하\AppData\Local\Temp\ipykernel_16300\4207078778.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)


tensor([[0]], dtype=torch.int32)


C:\Users\이민하\AppData\Local\Temp\ipykernel_16300\4207078778.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)


tensor([[0]], dtype=torch.int32)


C:\Users\이민하\AppData\Local\Temp\ipykernel_16300\4207078778.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)


...

tensor([[1]], dtype=torch.int32)


C:\Users\이민하\AppData\Local\Temp\ipykernel_16300\4207078778.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)


tensor([[0]], dtype=torch.int32)


C:\Users\이민하\AppData\Local\Temp\ipykernel_16300\4207078778.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)


tensor([[0]], dtype=torch.int32)
